In [1]:
from helpers.inference import run_inference
from helpers.download import download_row
import geopandas as gpd
from pathlib import Path
from tqdm.auto import tqdm
from threading import Thread
from concurrent.futures import ThreadPoolExecutor, as_completed
import gc

In [2]:
model_path = Path("models/regnety_002_v1.29_model.pkl")
working_dir = Path(
    "/Users/nick/Desktop/CL test"
    # "/media/nick/SNEAKERNET/Aus working v7"
)
model_path.exists(), working_dir.exists()
working_dir.mkdir(exist_ok=True)

In [3]:
grid_file = Path("data/Senntinele 2 grid coastal Aus.gpkg")
grid_gdf = gpd.read_file(grid_file)

In [4]:
tide_key_path = Path("world_tide_key.txt")
tide_key_path.exists()

True

In [5]:
extract_start_year = 2022
extract_end_year = 2022

In [6]:
# failed = []
# with ThreadPoolExecutor(max_workers=2) as executor:
#     # Submit download tasks
#     future_to_id = {
#         executor.submit(
#             download_row, row, tide_key_path, extract_start_year, extract_end_year
#         ): id
#         for id, row in grid_gdf.iterrows()
#         if not (working_dir / f"{row['Name']}_2022_2022_pred.tif").exists()
#     }

#     for future in tqdm(as_completed(future_to_id), total=len(future_to_id)):
#         id = future_to_id[future]
#         result = future.result()

#         if result is None:
#             failed.append(id)
#             print(f"Failed on {id}")
#         else:
#             bands, profile = result
#             export_path = working_dir / f"{grid_gdf.loc[id]['Name']}_2022_2022_pred.tif"
#             # Run inference sequentially
#             run_inference(model_path, export_path, bands, profile)
#             del bands
#             del profile
#             del result
#             gc.collect()

In [7]:
failed = []
inf_thread = Thread()
for id, row in tqdm(grid_gdf.iterrows(), total=len(grid_gdf)):
    # name = row["Name"]
    # if name != "52LDH":
    # continue
    export_path = working_dir / f"{row['Name']}_2022_2022_pred.tif"
    if export_path.exists():
        continue
    try:
        bands, profile = download_row(
            row,
            tide_key_path,
            extract_start_year,
            extract_end_year,
        )
        if inf_thread.is_alive():
            inf_thread.join()
        if bands is None:
            failed.append(id)
            print(f"Failed on {id}")
            continue
        inf_thread = Thread(
            target=run_inference,
            args=(
                model_path,
                export_path,
                bands,
                profile,
            ),
        )
        inf_thread.start()

    except Exception as e:
        print(e)
        failed.append(id)
        print(f"Failed on {id}")
        continue

  0%|          | 0/298 [00:00<?, ?it/s]

Downloading:   0%|          | 0/24 [00:00<?, ?it/s]

Downloading:   0%|          | 0/12 [00:00<?, ?it/s]

Exception in thread Thread-66 (run_inference):
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/c2m_prod/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/c2m_prod/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/nick/Documents/Projects/S2Coastline/helpers/inference.py", line 197, in run_inference
    pickle.load(
  File "/opt/homebrew/Caskroom/miniforge/base/envs/c2m_prod/lib/python3.11/site-packages/torch/storage.py", line 337, in _load_from_bytes
    return torch.load(io.BytesIO(b))
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/c2m_prod/lib/python3.11/site-packages/torch/serialization.py", line 1028, in load
    return _legacy_load(opened_file, map_location, pickle_module, **pickle_load_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^